In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import mrob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from scipy.optimize import minimize
import functools
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

#1. Straight line

In [ ]:
data_straight = pd.read_csv("acceleration_along_straigt_line.csv")

In [ ]:
class IMU:
    def __init__(self, start_R, start_v, start_p, g=9.81):
        self.R = start_R
        self.v = start_v
        self.p = start_p
        # self.g = np.array([0, 0, g])
        self.R_history, self.v_history, self.p_history = [], [], []

    def integrate(self, omega, a, dt):
        rotation = mrob.geometry.SO3(omega * dt)
        R = self.R.mul(rotation)
        v = self.v + self.R.R() @ (a * dt) # + self.g * dt 
        p = self.p + self.v * dt + 1/2 * self.R.R() @ (a * (dt ** 2)) #  1/2 * self.g * (dt ** 2)
        self.R = R 
        self.v = v 
        self.p = p
        self.R_history.append(R)
        self.v_history.append(v)
        self.p_history.append(p)

        return R, v, p

    def get_history(self):
        return self.R_history, self.v_history, self.p_history


In [ ]:
dt = np.ones(data_straight.shape[0]) * 0.005
angle_v = np.array(data_straight[['yaw', 'pitch', 'roll']])
angle_a = np.array(data_straight[['acc_x', 'acc_y', 'acc_z']])
gt_positions = np.array(data_straight[['x', 'y', 'z']])
gt_quat = np.array(data_straight[['yaw', 'pitch', 'roll', 'z']])

In [ ]:
imu_straight = IMU(mrob.geometry.SO3(np.eye(3)), angle_v[0], np.zeros(3))
for i in range(angle_v.shape[0]):
    imu_straight.integrate(angle_v[i], angle_a[i], dt[i])

In [ ]:
from numpy.linalg import norm

error = norm(np.array(data_straight['x']) - np.array(imu_straight.p_history)[:, 0])/np.array(data_straight['x']).shape[0]

MSE: 

In [ ]:
np.sqrt(error)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(np.array(imu_straight.p_history)[:, 0][:1000], linewidth=3, label="estimated trajectory");
plt.title(r"Straight trajectory estimation ($\epsilon = {}$)".format(np.round(error, 3)))
plt.plot(data_straight['x'][:1000], linewidth=1, label="ground trajectory");
plt.xlabel(r"$x$");
plt.ylabel(r"$y$");
plt.legend();

# 2. Circular line

In [ ]:
data_circle = pd.read_csv("acceleration_along_circle.csv")

In [ ]:
data_circle

In [ ]:
dt = np.ones(data_circle.shape[0]) * 0.005
angle_v = np.array(data_circle[['omega_x', 'omega_y', 'omega_z']])
angle_a = np.array(data_circle[['acc_x', 'acc_y', 'acc_z']])

In [ ]:
imu_circular = IMU(mrob.geometry.SO3(np.eye(3)), np.array(data_circle[['omega_x', 'omega_y', 'omega_z']].iloc[0]), np.array(data_circle[['x', 'y', 'z']].iloc[0]))
for i in range(angle_v.shape[0]):
    imu_circular.integrate(angle_v[i], angle_a[i], dt[i])

Rs = [R.R() for R in imu_circular.R_history]
ps = imu_circular.p_history = np.array(imu_circular.p_history)

In [ ]:
plt.plot(data_circle.x)

In [ ]:
plt.plot(ps[:, 0], ps[:, 1])

In [ ]:
ps

In [ ]:
import plotly.express as px



df = pd.DataFrame(ps,columns=['x','y','z'])

fig = px.line_3d(data_frame=df,x='x',y='y',z='z',height=1100, title='Intgrated circle trajectory')
fig.show()

In [ ]:
import sys
sys.path.append(".")

from uncertainty import sigma_visualize_3d, get_mc

In [ ]:
T_0 = mrob.geometry.SE3()
sigma_0 = np.diag([0.1,0.1,0.1,0.1,0.1,0.1])
mean = np.zeros(6)


axes, circumferences = sigma_visualize_3d(T_0,sigma_0,N = 100, K = 3)

df = pd.DataFrame(columns=['x','y','z'])

for key,val in axes.items():
    tmp = pd.DataFrame(val,columns=['x','y','z'])
    tmp['label'] = key
    df = pd.concat([df,tmp])

for key,val in circumferences.items():
    tmp = pd.DataFrame(val,columns=['x','y','z'])
    tmp['label'] = key
    df = pd.concat([df,tmp])

fig = px.line_3d(data_frame=df,x='x',y='y',z='z',color='label',hover_name='label',height=800,title='6D Pose Uncertainty Projected into 3D space.')

poses, xi = get_mc(T_0, sigma_0, mean,N=1_000)

particles = pd.DataFrame(poses, columns=['x','y','z'])

fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})

fig.show()

In [ ]:
T_0 = mrob.geometry.SE3([0.5,0.0,np.pi,10,0.1,-0.3])
sigma_0 = np.diag([0.3,0.01,0.31,0.4,0.4,0.1])


mean = np.zeros(6)


axes, circumferences = sigma_visualize_3d(T_0,sigma_0,N = 100, K = 3)

df = pd.DataFrame(columns=['x','y','z'])

for key,val in axes.items():
    tmp = pd.DataFrame(val,columns=['x','y','z'])
    tmp['label'] = key
    df = pd.concat([df,tmp])

for key,val in circumferences.items():
    tmp = pd.DataFrame(val,columns=['x','y','z'])
    tmp['label'] = key
    df = pd.concat([df,tmp])

fig = px.line_3d(data_frame=df,x='x',y='y',z='z',color='label',hover_name='label',height=1000,title='6D Pose Uncertainty Projected into 3D space')

poses, xi = get_mc(T_0, sigma_0, mean,N=1_0000)

particles = pd.DataFrame(poses, columns=['x','y','z'])

fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3,'color':"rgb(155,25,53)"})
# fig.update_layout(scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1),
#                               xaxis = dict(nticks=4, range=[-12,12],),
#                  yaxis = dict(nticks=4, range=[-12,12],),
#                  zaxis = dict(nticks=4, range=[-12,12],)))
fig.update_layout(scene = dict(aspectmode='data', aspectratio=dict(x=1, y=1, z=1)))

fig.show()

In [ ]:
T_0 = mrob.geometry.SE3([0.5,1.0,1.0,10,0.1,-0.3])
sigma_0 = np.diag([0.3,0.01,0.01,0.01,0.1,0.01])


mean = np.zeros(6)


axes, circumferences = sigma_visualize_3d(T_0,sigma_0,N = 100, K = 3)

df = pd.DataFrame(columns=['x','y','z'])

for key,val in axes.items():
    tmp = pd.DataFrame(val,columns=['x','y','z'])
    tmp['label'] = key
    df = pd.concat([df,tmp])

for key,val in circumferences.items():
    tmp = pd.DataFrame(val,columns=['x','y','z'])
    tmp['label'] = key
    df = pd.concat([df,tmp])

fig = px.line_3d(data_frame=df,x='x',y='y',z='z',color='label',hover_name='label',height=1000,title='6D Pose Uncertainty Projected into 3D space')

poses, xi = get_mc(T_0, sigma_0, mean,N=1_0000)

particles = pd.DataFrame(poses, columns=['x','y','z'])

fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3,'color':"rgb(255,255,53)"})
# fig.update_layout(scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1),
#                               xaxis = dict(nticks=4, range=[-12,12],),
#                  yaxis = dict(nticks=4, range=[-12,12],),
#                  zaxis = dict(nticks=4, range=[-12,12],)))
fig.update_layout(scene = dict(aspectmode='data', aspectratio=dict(x=1, y=1, z=1)))

fig.show()

In [ ]:
def ellipsoid_dataframe(T,sigma,N = 100, K = 1):
    axes, circumferences = sigma_visualize_3d(T=T,sigma=sigma,N = N, K = K)

    df = pd.DataFrame(columns=['x','y','z'])

    for key,val in axes.items():
        tmp = pd.DataFrame(val,columns=['x','y','z'])
        tmp['label'] = key
        df = pd.concat([df,tmp])

    for key,val in circumferences.items():
        tmp = pd.DataFrame(val,columns=['x','y','z'])
        tmp['label'] = key
        df = pd.concat([df,tmp])
    return df

def mc_dataframe(T, sigma, mean=np.zeros(6),N=1_00):
    poses, xi = get_mc(T=T, sigma=sigma, mean=mean,N=N)
    
    particles = pd.DataFrame(poses, columns=['x','y','z'])
    
    return particles
    

In [ ]:
# from uncertainty import compound_2nd
# from mrob.geometry import SE3, SO3

# def propagate(ps,Rs,sigma0, length):
    
#     vis_step = 500
#     compound_step=100
    
#     traj = pd.DataFrame(ps[:length],columns=['x','y','z'])
#     fig = px.line_3d(traj,x='x',y='y',z='z',height=900)
# #     fig.update_layout(scene = dict(
# #                      ,))
#     fig.update_layout(scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1),
#                                   xaxis = dict(nticks=4, range=[-10,10],),
#                      yaxis = dict(nticks=4, range=[-10,10],),
#                      zaxis = dict(nticks=4, range=[-10,10],)))
    
#     T = mrob.geometry.SE3()
#     mean = np.zeros(6)
    
#     sigma=sigma0
    
#     particles = mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[0]

#     fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})
    
#     wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[0]
#     fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
    
    
#     for i in range(len(ps[:length])-1):
        
#         if i % compound_step == 0:
# #             tmp = SE3(SO3(Rs[i]),ps[i]).inv().mul(SE3(SO3(Rs[i+1]),ps[i+1]))
#             tmp = (SE3(SO3(Rs[i+1]),ps[i+1]).inv()).mul(SE3(SO3(Rs[i]),ps[i]))

#             T, sigma = compound_2nd(T,sigma,tmp, np.diag([0.1,0.1,0,0.01,0.01,0.005]))
        
#         if i % vis_step == 0:
#             particles = mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[i]

#             fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})

#             wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[i]
#             fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
            
        
#     fig.update_yaxes(
#     scaleanchor = "x",
#     scaleratio = 1,)  
    
    
    
#     fig.show()
    
    

## Propagation along circle

In [ ]:
from uncertainty import compound_2nd
from mrob.geometry import SE3, SO3

def propagate(ps,Rs,sigma0, start, finish):
    
    vis_step = 250
#     compound_step=5
    
    traj = pd.DataFrame(ps[start:finish],columns=['x','y','z'])
    fig = px.line_3d(traj,x='x',y='y',z='z',height=900)
#     fig.update_layout(scene = dict(
#                      ,))
    fig.update_layout(scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1),
                                  xaxis = dict(nticks=4, range=[-15,15],),
                     yaxis = dict(nticks=4, range=[-15,15],),
                     zaxis = dict(nticks=4, range=[-15,15],)))
    
    T = mrob.geometry.SE3()
    mean = np.zeros(6)
    
    sigma=sigma0
    
    particles = mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[0]

    fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})
    
    wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[0]
    fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
    
    
    for i in range(len(ps[start:finish])-1):
        
#         if i % compound_step == 0:
        tmp = SE3(SO3(Rs[i]),ps[i]).inv().mul(SE3(SO3(Rs[i+1]),ps[i+1]))
#             tmp = (SE3(SO3(Rs[i+1]),ps[i+1]).inv()).mul(SE3(SO3(Rs[i]),ps[i]))

        T, sigma = compound_2nd(T,sigma, tmp, 1e-4*np.diag([0.01,0.01,0.01,0.01,0.01,0.01]))
        
        if i % vis_step == 0:
            particles =  mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[i] - T.t()

            fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})

            wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[i] - T.t()
            fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
            
        
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,)  
    
    
    
    fig.show()
    
    

In [ ]:
sigma_0 = np.diag([0.01,0.01,0.01,0.01,0.01,0.01])


propagate(ps,Rs,sigma_0, 2000, 7000)

## Propagating along straight line

In [ ]:
from uncertainty import compound_2nd
from mrob.geometry import SE3, SO3

def propagate(ps,Rs,sigma0, start, finish):
    
    vis_step = 250
#     compound_step=5
    
    traj = pd.DataFrame(ps[start:finish],columns=['x','y','z'])
    fig = px.line_3d(traj,x='x',y='y',z='z',height=900)
#     fig.update_layout(scene = dict(
#                      ,))
    fig.update_layout(scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1),
                                  xaxis = dict(nticks=4, range=[-15,15],),
                     yaxis = dict(nticks=4, range=[-15,15],),
                     zaxis = dict(nticks=4, range=[-15,15],)))
    
    T = mrob.geometry.SE3()
    mean = np.zeros(6)
    
    sigma=sigma0
    
    particles = mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[0]

    fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})
    
    wframe = ellipsoid_dataframe(T,sigma,N=1000,K=3)[['x','y','z']] + traj.iloc[0]
    fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
    
    
    for i in range(len(ps[start:finish])-1):
        
#         if i % compound_step == 0:
        tmp = SE3(SO3(Rs[i]),ps[i]).inv().mul(SE3(SO3(Rs[i+1]),ps[i+1]))
#             tmp = (SE3(SO3(Rs[i+1]),ps[i+1]).inv()).mul(SE3(SO3(Rs[i]),ps[i]))

        T, sigma = compound_2nd(T,sigma, tmp, 1e-4*np.diag([0.01,0.01,0.01,0.01,0.01,0.01]))
        
        if i % vis_step == 0:
            particles =  mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[i] - T.t()

            fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})

            wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[i] - T.t()
            fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
            
        
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,)  
    
    
    
    fig.show()
    
    

In [ ]:
Rline = [R.R() for R in imu_straight.R_history]
pline = imu_straight.p_history = np.array(imu_straight.p_history)

In [ ]:
sigma_0 = np.diag([0.1,0.01,0.01,0.05,0.05,0.05])

length = 2000

propagate(pline,Rline,sigma_0, 0, 4000)

In [ ]:
!ls

## Real world data

In [ ]:
df = pd.read_csv('data1.csv')

In [ ]:
from uncertainty import compound_2nd
from mrob.geometry import SE3, SO3

def propagate(df,sigma0, start, finish):
    
    vis_step = 350
#     compound_step=5
    
    traj = pd.DataFrame(df[['p1', 'p2', 'p3']].values[start:finish],columns=['x','y','z'])
    traj.values[:,2] = 0
#     print(traj.values)
    ps = traj.values
    
    Rs = df[['R11', 'R12', 'R13', 'R21', 'R22', 'R23', 'R31', 'R32','R33']].values.reshape((len(df),3,3))
#     print(Rs.shape)
    sigmas = df[['sigma_yaw','sigma_pitch', 'sigma_roll', 'sigma_p1', 'sigma_p2', 'sigma_p']].values
    
#     print(sigmas.shape)
    
    fig = px.line_3d(traj,x='x',y='y',z='z',height=900)
#     fig.update_layout(scene = dict(
#                      ,))
#     fig.update_layout(scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1),
#                                   xaxis = dict(nticks=4, range=[-100,100],),
#                      yaxis = dict(nticks=4, range=[-100,100],),
#                      zaxis = dict(nticks=4, range=[-100,100],)))
    
    T = mrob.geometry.SE3()
    mean = np.zeros(6)
    
    sigma=sigma0
    
    particles = mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[0]

    fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})
    
    wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[0]
    fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
    
    
    for i in range(len(ps[start:finish])-1):
        
#         if i % compound_step == 0:
        tmp = SE3(SO3(Rs[i]),ps[i]).inv().mul(SE3(SO3(Rs[i+1]),ps[i+1]))
#             tmp = (SE3(SO3(Rs[i+1]),ps[i+1]).inv()).mul(SE3(SO3(Rs[i]),ps[i]))

        T, sigma = compound_2nd(T,sigma, tmp, 1e-4*np.diag(sigmas[i]))
        
        if i % vis_step == 0:
            particles =  mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[i] - T.t()

            fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})

            wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[i] - T.t()
            fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
            
        
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,)  
    
    
    
    fig.show()
    

In [ ]:
sigma_0 = np.diag([0.1,0.01,0.01,0.05,0.05,0.05])

length = 2000

propagate(df,sigma_0, 0, len(df))

In [ ]:
df = pd.read_csv('data2.csv')

from uncertainty import compound_2nd
from mrob.geometry import SE3, SO3

def propagate(df,sigma0, start, finish):
    
    vis_step = 350
#     compound_step=5
    
    traj = pd.DataFrame(df[['p1', 'p2', 'p3']].values[start:finish],columns=['x','y','z'])
    traj.values[:,2] = 0
#     print(traj.values)
    ps = traj.values
    
    Rs = df[['R11', 'R12', 'R13', 'R21', 'R22', 'R23', 'R31', 'R32','R33']].values.reshape((len(df),3,3))
#     print(Rs.shape)
    sigmas = df[['sigma_yaw','sigma_pitch', 'sigma_roll', 'sigma_p1', 'sigma_p2', 'sigma_p']].values
    
#     print(sigmas.shape)
    
    fig = px.line_3d(traj,x='x',y='y',z='z',height=900)
#     fig.update_layout(scene = dict(
#                      ,))
#     fig.update_layout(scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1),
#                                   xaxis = dict(nticks=4, range=[-100,100],),
#                      yaxis = dict(nticks=4, range=[-100,100],),
#                      zaxis = dict(nticks=4, range=[-100,100],)))
    
    T = mrob.geometry.SE3()
    mean = np.zeros(6)
    
    sigma=sigma0
    
    particles = mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[0]

    fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})
    
    wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[0]
    fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
    
    
    for i in range(len(ps[start:finish])-1):
        
#         if i % compound_step == 0:
        tmp = SE3(SO3(Rs[i]),ps[i]).inv().mul(SE3(SO3(Rs[i+1]),ps[i+1]))
#             tmp = (SE3(SO3(Rs[i+1]),ps[i+1]).inv()).mul(SE3(SO3(Rs[i]),ps[i]))

        T, sigma = compound_2nd(T,sigma, tmp, 1e-4*np.diag(sigmas[i]))
        
        if i % vis_step == 0:
            particles =  mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[i] - T.t()

            fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})

            wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[i] - T.t()
            fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
            
        
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,)  
    
    
    
    fig.show()
    

sigma_0 = np.diag([0.1,0.01,0.01,0.05,0.05,0.5])

length = 2000

propagate(df,sigma_0, 0, len(df))

In [ ]:
df = pd.read_csv('data3.csv')

from uncertainty import compound_2nd, compound_4th
from mrob.geometry import SE3, SO3

def propagate(df,sigma0, start, finish):
    
    vis_step = 500
#     compound_step=5
    
    traj = pd.DataFrame(df[['p1', 'p2', 'p3']].values[start:finish],columns=['x','y','z'])
    traj.values[:,2] = 0
#     print(traj.values)
    ps = traj.values
    
    Rs = df[['R11', 'R12', 'R13', 'R21', 'R22', 'R23', 'R31', 'R32','R33']].values.reshape((len(df),3,3))
#     print(Rs.shape)
    sigmas = df[['sigma_yaw','sigma_pitch', 'sigma_roll', 'sigma_p1', 'sigma_p2', 'sigma_p']].values
    
#     print(sigmas.shape)
    
    fig = px.line_3d(traj,x='x',y='y',z='z',height=900)
#     fig.update_layout(scene = dict(
#                      ,))
    fig.update_layout(scene = dict(aspectmode='data', aspectratio=dict(x=1, y=1, z=1)))
#                                   xaxis = dict(nticks=4, range=[-100,100],),
#                      yaxis = dict(nticks=4, range=[-100,100],),
#                      zaxis = dict(nticks=4, range=[-100,100],)))
    
    T = mrob.geometry.SE3()
    mean = np.zeros(6)
    
    sigma=sigma0
    
    particles = mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[0]

    fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})
    
    wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[0]
    fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
    
    
    for i in range(len(ps[start:finish])-1):
        
#         if i % compound_step == 0:
        tmp = SE3(SO3(Rs[i]),ps[i]).inv().mul(SE3(SO3(Rs[i+1]),ps[i+1]))
#             tmp = (SE3(SO3(Rs[i+1]),ps[i+1]).inv()).mul(SE3(SO3(Rs[i]),ps[i]))

        T, sigma = compound_2nd(T,sigma, tmp, 1e-6*np.diag(sigmas[i]))
        
        if i % vis_step == 0:
            particles =  mc_dataframe(T,sigma,mean,N=1000) + traj.iloc[i] - T.t()

            fig.add_scatter3d(x=particles['x'],y=particles['y'],z=particles['z'],opacity=0.5,mode='markers',marker={'size':3})

            wframe = ellipsoid_dataframe(T,sigma,N=100,K=3)[['x','y','z']] + traj.iloc[i] - T.t()
            fig.add_scatter3d(x=wframe['x'],y = wframe['y'],z=wframe['z'],mode='lines')
            
        
    fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,)  
    
    
    
    fig.show()
    

sigma_0 = np.diag([0.1,0.01,0.01,0.05,0.05,0.005])

length = 2000

propagate(df,sigma_0, 0, len(df))